In [1]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [2]:
# Define the model
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [3]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
  W_sm_exp = dy.parameter(W_sm)
  b_sm_exp = dy.parameter(b_sm)
  return W_sm_exp * cbow + b_sm_exp  # computational graph上での * はちゃんとmatmul, dot的な感じで行列の積となってる

In [4]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5485, time=0.44s
iter 0: test acc=0.3851
iter 1: train loss/sent=1.1531, time=0.45s
iter 1: test acc=0.3855
iter 2: train loss/sent=0.8708, time=0.41s
iter 2: test acc=0.3932
iter 3: train loss/sent=0.6410, time=0.40s
iter 3: test acc=0.4000
iter 4: train loss/sent=0.4574, time=0.40s
iter 4: test acc=0.3891
iter 5: train loss/sent=0.3333, time=0.39s
iter 5: test acc=0.3873
iter 6: train loss/sent=0.2381, time=0.42s
iter 6: test acc=0.3837
iter 7: train loss/sent=0.1771, time=0.43s
iter 7: test acc=0.3837
iter 8: train loss/sent=0.1289, time=0.43s
iter 8: test acc=0.3878
iter 9: train loss/sent=0.1022, time=0.42s
iter 9: test acc=0.3760
iter 10: train loss/sent=0.0758, time=0.43s
iter 10: test acc=0.3760
iter 11: train loss/sent=0.0612, time=0.43s
iter 11: test acc=0.3665
iter 12: train loss/sent=0.0474, time=0.43s
iter 12: test acc=0.3719
iter 13: train loss/sent=0.0371, time=0.43s
iter 13: test acc=0.3629
iter 14: train loss/sent=0.0317, time=0.44s
iter 14: t